# Import module

In [3]:
import os
from pyvi import ViTokenizer
import re

import tensorflow as tf
import numpy as np

import pickle
from tensorflow.keras.models import load_model

In [40]:
LOOKBACK_LENGTH = 50

# Làm sạch đầu vào

In [89]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def make_clean(string, to_tokens=True):
    res = string.lower()
    res = remove_emoji(res)
    res = re.sub(r'[!“”"#$%&\()*+,-./:;<=>?@[\]^_`{|}~]', " ", res)
    res = re.sub(r'\s+', ' ', res).strip()
    res = ViTokenizer.tokenize(res)
    if to_tokens:
        res = res.split()
    return res

make_clean('Tôi     là(*(*%$sinh (*&viên', to_tokens=True)

['tôi', 'là', 'sinh_viên']

# Hiện thực hóa model

In [90]:
# tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
# load weight model lstm
model = load_model('weight_lstm_model.h5', compile=False)

In [95]:
# input đầu vào
text = 'TPHCM đào tạo đại học cho người sau cai nghiện Đại học Mở-Bán công đã khai giảng lớp đại học từ xa khoa Xã hội và Nhân văn đầu tiên cho học viên, người sau cai nghiện và một số cán bộ, công nhân viên các trung tâm giáo dục dạy nghề của thành phố. Lớp học, được đào tạo theo chế độ tín chỉ, gồm 169 sinh viên, trong đó có 47 học viên sau cai nghiện. Các sinh viên sẽ học tập trung mỗi tuần 2 ngày, thời gian còn lại là tự nghiên cứu. Các học viên sau khi tốt nghiệp đại học sẽ'

In [96]:
def process_input(text):
    # làm sạch đầu vào
    clean_text = make_clean(text, to_tokens=True)
    # lấy ra 50 từ cuối 
    last_seq = [clean_text[-50:]]
    # chuyển 50 từ cuối đó thành số
    tokens = tokenizer.texts_to_sequences(last_seq)
    seq_len = len(tokens[0])
    pad_len = [0 for _ in range(LOOKBACK_LENGTH-seq_len)]
    return [pad_len + tokens[0]]

def predict_class(text): # text đã qua tiền xử lý
    text_pred = model.predict(text, verbose=None) 
    return np.argmax(text_pred, axis=1)

# tạo các từ tiếp theo dựa trên input đầu vào
def generate(text, generate_len=15): # 15 từ sẽ được tạo 
    print('text input:', text)
    text = process_input(text)
    predict_seq = []
    for _ in range(generate_len):
        pred = predict_class(text)
        predict_seq.append(pred[0])
        text = np.concatenate([text, [pred]], axis=1)[:,1:]
    print('text generated:', tokenizer.sequences_to_texts([predict_seq])[0].replace('_', ' '))

In [97]:
generate(text)

text input: TPHCM đào tạo đại học cho người sau cai nghiện Đại học Mở-Bán công đã khai giảng lớp đại học từ xa khoa Xã hội và Nhân văn đầu tiên cho học viên, người sau cai nghiện và một số cán bộ, công nhân viên các trung tâm giáo dục dạy nghề của thành phố. Lớp học, được đào tạo theo chế độ tín chỉ, gồm 169 sinh viên, trong đó có 47 học viên sau cai nghiện. Các sinh viên sẽ học tập trung mỗi tuần 2 ngày, thời gian còn lại là tự nghiên cứu. Các học viên sau khi tốt nghiệp đại học sẽ
text generated: được tuyển dụng làm việc tại một công ty trường cao đẳng với sự hỗ trợ của trường chương trình đào tạo
